In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Bethel,US,2021-07-14 22:16:22,41.3712,-73.4140,86.85,77,40,6.91
4,4,Butaritari,KI,2021-07-14 22:11:48,3.0707,172.7902,83.62,77,5,12.68
7,7,Bengkulu,ID,2021-07-14 22:12:08,-3.8004,102.2655,75.16,70,100,3.18
10,10,Thompson,CA,2021-07-14 22:15:08,55.7435,-97.8558,84.94,20,26,3.94
14,14,Avarua,CK,2021-07-14 22:12:43,-21.2078,-159.7750,80.65,74,40,11.50
21,21,Ucluelet,CA,2021-07-14 22:10:09,48.9329,-125.5528,84.04,86,77,1.01
24,24,Nishihara,JP,2021-07-14 22:13:56,26.1842,127.7558,83.48,81,75,8.05
25,25,Jamestown,US,2021-07-14 22:12:13,42.0970,-79.2353,79.02,56,1,11.99
26,26,Kapaa,US,2021-07-14 22:16:34,22.0752,-159.3190,84.18,82,90,3.00
28,28,Padang,ID,2021-07-14 22:13:50,-0.9492,100.3543,75.33,71,96,4.12


In [9]:
preferred_cities_df.count()

City_ID       234
City          234
Country       234
Date          234
Lat           234
Lng           234
Max Temp      234
Humidity      234
Cloudiness    234
Wind Speed    234
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bethel,US,86.85,41.3712,-73.4140,
4,Butaritari,KI,83.62,3.0707,172.7902,
7,Bengkulu,ID,75.16,-3.8004,102.2655,
10,Thompson,CA,84.94,55.7435,-97.8558,
14,Avarua,CK,80.65,-21.2078,-159.7750,
21,Ucluelet,CA,84.04,48.9329,-125.5528,
24,Nishihara,JP,83.48,26.1842,127.7558,
25,Jamestown,US,79.02,42.0970,-79.2353,
26,Kapaa,US,84.18,22.0752,-159.3190,
28,Padang,ID,75.33,-0.9492,100.3543,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Bethel,US,86.85,41.3712,-73.4140,Courtyard by Marriott Danbury
4,Butaritari,KI,83.62,3.0707,172.7902,Isles Sunset Lodge
7,Bengkulu,ID,75.16,-3.8004,102.2655,Grage Hotel Bengkulu
10,Thompson,CA,84.94,55.7435,-97.8558,Burntwood Hotel
14,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...
591,Aloleng,PH,81.01,16.1307,119.7824,Steve Walden and Emma Sison-Walden Residence
592,Buena Vista,US,85.06,43.4203,-83.8986,Red Roof Inn Saginaw - Frankenmuth
595,Alofi,NU,82.29,-19.0595,-169.9187,Taloa Heights
596,Chabahar,IR,86.85,25.2919,60.6430,هتل گدروشیا


In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))